In [1]:
import itertools
import math


class ImpossibleTriangleException(Exception): pass

    
def initialize_path(first_wall, axis, direction, dimensions, walls):
    current_wall = first_wall
    axis = axis
    on_axis = first_wall % 2 == axis
    on_opposite_axis = True
    along_axis_length = dimensions[0] if axis == 0 else dimensions[1]
    against_axis_length = dimensions[1] if axis == 0 else dimensions[0]
    first_adjacent_distance = walls[first_wall][0]
    max_first_opposite_distance = walls[(first_wall + direction) % 4][0]
    opposite_axis_distance = max_first_opposite_distance
    adjacent_axis_distance = first_adjacent_distance
    opposite_axis_length = along_axis_length if on_opposite_axis else against_axis_length
    adjacent_axis_length = against_axis_length if on_opposite_axis else along_axis_length
    count = 1

    return (
        current_wall,                   # 0
        direction,                      # 1
        on_axis,                        # 2
        on_opposite_axis,               # 3
        opposite_axis_distance,         # 4
        adjacent_axis_distance,         # 5
        opposite_axis_length,           # 6
        adjacent_axis_length,           # 7
        first_adjacent_distance,        # 8
        max_first_opposite_distance,    # 9
        count                           # 10
    )


def turn(path):
    direction = path[1]
    on_axis = -path[2]
    on_opposite_axis = -path[3]
    current_wall = (path[0] + direction) % 4
    opposite_axis_distance = path[4] + on_opposite_axis * path[6]
    adjacent_axis_distance = path[5] + (not on_opposite_axis) * path[7]
    opposite_axis_length = path[6]
    adjacent_axis_length = path[7]
    first_adjacent_distance = path[8]
    max_first_opposite_distance = path[9]
    count = path[10] + 1
    
    return (
        current_wall,
        direction,
        on_axis,
        on_opposite_axis,
        opposite_axis_distance,
        adjacent_axis_distance,
        opposite_axis_length,
        adjacent_axis_length,
        first_adjacent_distance,
        max_first_opposite_distance,
        count
    )

    
def reflect(path):
    direction = -path[1]
    on_axis = path[2]
    on_opposite_axis = path[3]
    current_wall = (path[0] + 2) % 4
    opposite_axis_distance = path[4] + (not on_opposite_axis) * path[6]
    adjacent_axis_distance = path[5] + on_opposite_axis * path[7]
    opposite_axis_length = path[6]
    adjacent_axis_length = path[7]
    first_adjacent_distance = path[8]
    max_first_opposite_distance = path[9]
    count = path[10] + 1

    return (
        current_wall,
        direction,
        on_axis,
        on_opposite_axis,
        opposite_axis_distance,
        adjacent_axis_distance,
        opposite_axis_length,
        adjacent_axis_length,
        first_adjacent_distance,
        max_first_opposite_distance,
        count
    )


def extend_path(path):
    on_axis = path[2]
    if on_axis: return [turn(path), reflect(path)]
    else: return [turn(path)]


def calc_distance(path, walls):
    current_wall = path[0]
    direction = path[1]
    on_axis = path[2]
    on_opposite_axis = path[3]
    opposite_axis_distance = path[4]
    adjacent_axis_distance = path[5]
    first_adjacent_distance = path[8]
    max_first_opposite_distance = path[9]
    count = path[10]

    if on_opposite_axis:
        opposite_axis_distance -= walls[(current_wall - direction) % 4][1]
        adjacent_axis_distance += walls[current_wall][1]

    if not on_opposite_axis:
        opposite_axis_distance += walls[current_wall][1]
        adjacent_axis_distance -= walls[(current_wall + direction) % 4][1]

    first_opposite_distance = opposite_axis_distance / adjacent_axis_distance * first_adjacent_distance
    first_hypotenuse_distance = math.sqrt(first_adjacent_distance ** 2 + first_opposite_distance ** 2)
    beam_distance = opposite_axis_distance / first_opposite_distance * first_hypotenuse_distance

    if opposite_axis_distance == 0 or adjacent_axis_distance == 0:
        raise ImpossibleTriangleException

    if first_opposite_distance > max_first_opposite_distance:
        raise ImpossibleTriangleException

    return beam_distance

In [ ]:

def solution(dimensions, my_position, trainer_position, distance):
    
    WALLS = (
        (dimensions[1] - my_position[1], dimensions[1] - trainer_position[1]),
        (dimensions[0] - my_position[0], dimensions[0] - trainer_position[0]),
        (my_position[1], trainer_position[1]),
        (my_position[0], trainer_position[0])
    )

    result = 0
    queue = [initialize_path(*product, dimensions, WALLS) for product in itertools.product((0,1,2,3), (0,1), (-1,1))]

    while queue:
        path = queue.pop()
        try:
            if calc_distance(path, WALLS) <= distance:
                result += 1
                queue.extend(extend_path(path))
                
        except ImpossibleTriangleException:
            queue.extend(extend_path(path))


    return result

In [4]:
paths = solution([300,275], [150,150], [185,100], 500)

In [52]:
# return (
#         current_wall,               # 0
#         axis
#         direction,                  # 1
#         on_axis,                    # 2
#         on_opposite_axis,           # 3
#         opposite_axis_distance,     # 4
#         adjacent_axis_distance,     # 5
#         opposite_axis_length,       # 6
#         adjacent_axis_length,       # 7
#         first_adjacent_distance,    # 8
#     )

paths

[(0, 0, -1, True, True, 150, 125, 300, 275, 125),
 (0, 0, 1, True, True, 150, 125, 300, 275, 125),
 (0, 1, -1, False, True, 150, 125, 275, 300, 125),
 (0, 1, 1, False, True, 150, 125, 275, 300, 125),
 (1, 0, -1, False, True, 125, 150, 300, 275, 150),
 (1, 0, 1, False, True, 150, 150, 300, 275, 150),
 (1, 1, -1, True, True, 125, 150, 275, 300, 150),
 (1, 1, 1, True, True, 150, 150, 275, 300, 150),
 (2, 0, -1, True, True, 150, 150, 300, 275, 150),
 (2, 0, 1, True, True, 150, 150, 300, 275, 150),
 (2, 1, -1, False, True, 150, 150, 275, 300, 150),
 (2, 1, 1, False, True, 150, 150, 275, 300, 150),
 (3, 0, -1, False, True, 150, 150, 300, 275, 150),
 (3, 0, 1, False, True, 125, 150, 300, 275, 150),
 (3, 1, -1, True, True, 150, 150, 275, 300, 150),
 (3, 1, 1, True, True, 125, 150, 275, 300, 150)]